In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from typing import Optional
from collections import namedtuple

from scipy.spatial.transform import Rotation as R

from mcap_protobuf.decoder import DecoderFactory
from mcap.reader import make_reader

from ros2_calib import ros_utils, bag_handler
from ros2_calib.dataload.calib_manager_handler import CalibManagerHandler

In [2]:
fx, fy, cx, cy = [1051.072979879756, 1051.157809600071, 630.1033101376631, 346.2213367453403]
K = [
    fx, 0.0, cx,
    0.0, fy, cy,
    0.0, 0.0, 1.0,
]
D = [-0.4169764302228503, 0.2032754719673925, 0.002265081048171727, -0.002142515738492578, 0, 0, 0, 0]

LIDAR_TOPIC = "/lidar_fc/cloud"
CAMERA_TOPIC = "/camera_fl/image"

FLU_TO_RDF = np.array(
    [[0, -1, 0, 0], [0, 0, -1, 0], [1, 0, 0, 0], [0, 0, 0, 1]], dtype=np.float64
)

BAG_READER = make_reader(
    open("../data/short_bag/short.mcap", "rb"), decoder_factories=[DecoderFactory()]
)
CALIB_HANDLER = CalibManagerHandler("../ur.autopilot/calib_manager.yaml")

[DEBUG] Correctly loaded calib_manager.yaml file.


In [3]:
def euler_to_transform(roll, pitch, yaw, x, y, z):
    rotation = R.from_euler("xyz", [roll, pitch, yaw])
    # rotation = R.from_euler("zyx", [yaw, pitch, roll])
    rot_matrix = rotation.as_matrix()

    transform = np.eye(4)
    transform[:3, :3] = rot_matrix
    transform[:3, 3] = [x, y, z]

    return transform

In [4]:
def transform_to_euler(transform):
    rotation = R.from_matrix(transform[:3, :3])
    roll, pitch, yaw = rotation.as_euler("xyz")
    x, y, z = transform[:3, 3]
    return roll, pitch, yaw, x, y, z

In [5]:
def find_transform_path(from_frame: str, to_frame: str, tf_tree=CALIB_HANDLER.tf_tree):
    def _get_all_tf_frames():
        frames = set(tf_tree.keys())
        for children in tf_tree.values():
            frames.update(children.keys())
        return sorted(list(frames))

    if from_frame == to_frame:
        return np.eye(4)
    if not tf_tree:
        return None

    from collections import deque

    q = deque([(from_frame, np.eye(4))])
    visited = {from_frame}

    adj = {frame: [] for frame in _get_all_tf_frames()}
    for p, children in tf_tree.items():
        for c, data in children.items():
            adj[p].append((c, data["transform"]))
            adj[c].append((p, np.linalg.inv(data["transform"])))

    while q:
        curr_frame, T = q.popleft()
        if curr_frame == to_frame:
            return T
        for neighbor, t in adj.get(curr_frame, []):
            if neighbor not in visited:
                visited.add(neighbor)
                q.append((neighbor, T @ t))
    return None

In [ ]:
from_frame = "lidar_fc"
to_frame = "camera_fl"
tf = find_transform_path(from_frame, to_frame)
roll, pitch, yaw, x, y, z = transform_to_euler(tf)
print(f"From '{from_frame}' to '{to_frame}'\nRPY: [{roll}, {pitch}, {yaw}]\nXYZ: [{x}, {y}, {z}]")

From 'lidar_fc' to 'camera_fl'
RPY: [-0.00015297005030224302, -0.00022463712454023899, -3.4310971632164044e-05]
XYZ: [0.4814902937513916, 0.20351937233648112, -0.013748588794860437]


In [14]:
np.set_printoptions(formatter={'float': '{: 0.5f}'.format})
print(tf)

[[ 1.00000  0.00003 -0.00022  0.48149]
 [-0.00003  1.00000  0.00015  0.20352]
 [ 0.00022 -0.00015  1.00000 -0.01375]
 [ 0.00000  0.00000  0.00000  1.00000]]


In [18]:
tf_inv = np.linalg.inv(tf)
roll, pitch, yaw, x, y, z = transform_to_euler(tf_inv)
print(f"From '{to_frame}' to '{from_frame}'\nRPY: [{roll: .5f}, {pitch: .5f}, {yaw: .5f}]\nXYZ: [{x: .5f}, {y: .5f}, {z: .5f}]")
print(tf_inv)

From 'camera_fl' to 'lidar_fc'
RPY: [ 0.00015,  0.00022,  0.00003]
XYZ: [-0.48148, -0.20354,  0.01383]
[[ 1.00000 -0.00003  0.00022 -0.48148]
 [ 0.00003  1.00000 -0.00015 -0.20354]
 [-0.00022  0.00015  1.00000  0.01383]
 [ 0.00000  0.00000  0.00000  1.00000]]


In [22]:
tf = euler_to_transform(-0.009157213491241987, -0.012959678550877582, -0.0035239750612062514, 0.5555989966011231, 0.23560272360118434, 0.020328317302382968)
tf_inv = np.linalg.inv(tf)
transform_to_euler(tf_inv)

(np.float64(0.009203593830534765),
 np.float64(0.012926782712663343),
 np.float64(0.003642801255058303),
 np.float64(-0.5549821447889566),
 np.float64(-0.23742891476015082),
 np.float64(-0.015312030501667549))

In [24]:
transform_to_euler(np.array([[ 0.99990982, -0.00352367,  0.01295932, -0.55498214],
       [ 0.00364249,  0.99995145, -0.00915632, -0.23742891],
       [-0.01292642,  0.00920269,  0.9998741 , -0.01531203],
       [ 0.        ,  0.        ,  0.        ,  1.        ]]))

(np.float64(0.009203593084814516),
 np.float64(0.012926783452783308),
 np.float64(0.00364280088511715),
 np.float64(-0.55498214),
 np.float64(-0.23742891),
 np.float64(-0.01531203))